# Capstone Project - Predicting 30-day Readmission for Diabetic Patients

## Step 1: Load and Preview Data

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("diabetic_data.csv")

# Create binary target variable: 1 if readmitted within 30 days, else 0
df["readmitted_binary"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)
df.head()

## Step 2: Clean and Prepare the Data

In [ ]:
# Drop unneeded columns and handle missing values
df_clean = df.drop(columns=["encounter_id", "patient_nbr", "weight", "payer_code", "medical_specialty", "readmitted"])
df_clean.replace("?", np.nan, inplace=True)
df_clean.dropna(inplace=True)

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder
label_cols = df_clean.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col])
    label_encoders[col] = le

df_clean.head()

## Step 3: Train Logistic Regression Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split data
X = df_clean.drop(columns=["readmitted_binary"])
y = df_clean["readmitted_binary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Step 4: Evaluate the Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Print classification report
print(classification_report(y_test, y_pred))

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Readmitted", "Readmitted"],
            yticklabels=["Not Readmitted", "Readmitted"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()